In [1]:
from skmultilearn.dataset import load_dataset
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import re
import random
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import os
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.dataset import load_from_arff
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, Subset
import itertools
import pickle
from sklearn.preprocessing import MinMaxScaler
from skmultilearn.dataset import load_dataset
from skmultilearn.model_selection import IterativeStratification
from sklearn.neighbors import NearestNeighbors
import pdb
import scipy
from sklearn.metrics import roc_auc_score
import warnings
import numpy as np
import pandas as pd
from enum import Enum
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import LambdaLR
from util import *
from layers import  *
from model import *
from function import *
from collections import deque
from scipy.stats import entropy
warnings.filterwarnings("ignore")

In [2]:
def seed_all(seed): 
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed=42
seed_all(seed)
device = torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")

In [3]:
class CFG:
    def __init__(self,name,X,y):
        self.name=name      
        self.X_train=X
        self.y_train=y
        self.configs={} 
    def getconfig(self):
        self.configs['label_matrix']=np.array(self.y_train)
        self.configs['num_classes']=self.y_train.shape[1] 
        self.configs['num_ins']=self.X_train.shape[0] 
        self.configs['seed']=42 
        self.configs['batch_size']=128
        self.configs['epoch']=100 
        self.configs['lr']=1e-4
        self.configs['device']=torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")
        self.configs['weight']=limb(np.array(self.X_train),np.array(self.y_train))
        self.configs['extra_sample']=int(self.X_train.shape[0]*0.1)
        self.configs['min_ins_idx']=minority_instance(np.array(self.y_train))
        self.configs['minority_label_indices'],_=Labeltype(np.array(self.y_train))
        self.configs['weight_list']=calweight(np.array(self.X_train),np.array(self.y_train))
        self.configs['card'],_=CardAndDens(np.array(self.X_train),np.array(self.y_train))
        #DELA
        if self.name=='DELA':
            self.configs['in_features']=self.X_train.shape[1] 
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)    
            self.configs['lr_ratio']=0.8
            self.configs['drop_ratio']=0.2
            self.configs['tau']=2/3
            self.configs['beta']=1e-4
            self.configs['out_index']=-1
        if self.name=='CLIF':
            self.configs['class_emb_size']=self.y_train.shape[1]  
            self.configs['input_x_size']=self.X_train.shape[1] 
            self.configs['num_layers']=2 
            self.configs['in_layers']=3 
            self.configs['hidden_list']=[math.ceil(self.y_train.shape[1]/2)]  
            self.configs['out_index']=0        
        if self.name=='PACA':
            self.configs['drop_ratio']=0.1
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)
            self.configs['in_features']=self.X_train.shape[1] #输入x的维度
            self.configs['rand_seed']=self.configs['seed']
            self.configs['eps']=1e-8    
            self.configs['lr_scheduler']='fix'
            self.configs['binary_data']=False
            self.configs['weight_decay']=1e-5
            self.configs['alpha']=2
            self.configs['gamma']=10
            self.configs['scheduler_warmup_epoch']=5
            self.configs['scheduler_decay_epoch']=10
            self.configs['scheduler_decay_rate']=1e-5 
            self.configs['out_index']=-2
        return self.configs

In [4]:
seed_all(seed)
device = torch.device('cuda')
def FeatureSelect(X,p):
    if p==1:
        return X.toarray(),feature_names
    else:
        featurecount=int(X.shape[1]*p)
        Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
        Allfeatureindex=[i for i in range(X.shape[1])]
        featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
        new_x=np.delete(X.toarray(),featureindex,axis=1)
        new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def LabelSelect(y):
    b=[]
    new_labelname=[i for i in label_names]
    for i in range(y.shape[1]):
        if y[:,i].sum()<=20:
            b.append(i)
            new_labelname.remove(label_names[i])
    new_y=np.delete(y.toarray(),b,axis=1)
    return new_y,new_labelname
def macro_averaging_auc(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(l)
    q = np.sum(Y, 0)

    zero_column_count = np.sum(q == 0)
#     print(f"all zero for label: {zero_column_count}")
    r, c = np.nonzero(Y)
    for i, j in zip(r, c):
        p[j] += np.sum((Y[ : , j] < 0.5) * (O[ : , j] <= O[i, j]))

    i = (q > 0) * (q < n)

    return np.sum(p[i] / (q[i] * (n - q[i]))) / l
def hamming_loss(Y, P, O):
    n = (Y.shape[0] + P.shape[0]) // 2
    l = (Y.shape[1] + P.shape[1]) // 2

    s1 = np.sum(Y, 1)
    s2 = np.sum(P, 1)
    ss = np.sum(Y * P, 1)

    return np.sum(s1 + s2 - 2 * ss) / (n * l)
def one_error(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2

    i = np.argmax(O, 1)

    return np.sum(1 - Y[range(n), i]) / n
def ranking_loss(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(n)
    q = np.sum(Y, 1)

    r, c = np.nonzero(Y)
    for i, j in zip(r, c): 
        p[i] += np.sum((Y[i, : ] < 0.5) * (O[i, : ] >= O[i, j]))

    i = (q > 0) * (q < l)

    return np.sum(p[i] / (q[i] * (l - q[i]))) / n
def micro_f1(Y, P, O):
    return f1_score(Y, P, average='micro')
def macro_f1(Y, P, O):
    return f1_score(Y, P, average='macro')
def eval_metrics(mod, metrics, datasets,batch_size,device):
    res_dict = {}
    mod.eval()
    y_true_list = []
    y_scores_list = []
    test_dataloader = DataLoader(datasets, batch_size=batch_size, shuffle=True, num_workers=0)
    for x, y in test_dataloader:
        _, y_pred=mod.predict(x)
#         pdb.set_trace()
        y_true_list.append(y.cpu().numpy())
        y_scores_list.append(y_pred.cpu().numpy())
    y_true = np.vstack(y_true_list)
    y_prob = np.vstack(y_scores_list)
    y_pred = np.round(y_prob).astype(int)
    res_dict1 = {metric.__name__: metric(y_true, y_pred,y_prob) for metric in metrics}
#         # Calculate metric.
#         res_dict1 = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:2]}
#         res_dict2 = {metric.__name__: metric(y_true, y_prob) for metric in metrics[2:5]}
#         res_dict1.update(res_dict2)
#     res_dict[f'dataset_{ix}']=res_dict1
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:8]}
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_prob) for metric in metrics[8:9]}
    return res_dict1

In [5]:
def Imbalance(y):
    countmatrix=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        countmatrix.append(count1)
    maxcount=max(countmatrix)
    ImbalanceRatioMatrix=[maxcount/i for i in countmatrix]
    MaxIR=max(ImbalanceRatioMatrix)
    MeanIR=sum(ImbalanceRatioMatrix)/len(ImbalanceRatioMatrix)
    return ImbalanceRatioMatrix,MeanIR,countmatrix
def Labeltype(y):
    # 计算不平衡率矩阵和平均不平衡率
    ImbalanceRatioMatrix, MeanIR, _ = Imbalance(y)
    
    # 计算每个标签与平均不平衡率的差值
    DifferenceImbalanceRatioMatrix = [i - MeanIR for i in ImbalanceRatioMatrix]
    
    MinLabelIndex = []  # 存储少数标签的索引
    MajLabelIndex = []  # 存储多数标签的索引
    count = 0
    
    # 根据差值将标签分类为少数标签或多数标签
    for i in DifferenceImbalanceRatioMatrix:
        if i > 0:
            MinLabelIndex.append(count)
        else:
            MajLabelIndex.append(count)
        count += 1

    # 根据索引获取少数和多数标签的名称
    MinLabelName = [label_names[i][0] for i in MinLabelIndex]
    MajLabelName = [label_names[i][0] for i in MajLabelIndex]
    
    # 创建字典，键是标签索引，值是标签名称
    MinLabeldic = dict(zip(MinLabelIndex, MinLabelName))
    MajLabeldic = dict(zip(MajLabelIndex, MajLabelName))
    
    return MinLabeldic, MajLabeldic

def Neighbor_Based(train_X, train_y, p):
    # 定义 LocalType 枚举类
    class LocalType(Enum):
        Safe = 1
        BoderLine = 2
        Rare = 3
        Outlier = 4
        Majority = 0

    np.random.seed(21)
    n_neighbors = 5
    # 计算最近邻
    nbs = NearestNeighbors(n_neighbors=n_neighbors + 1, metric='euclidean', algorithm='kd_tree').fit(train_X)
    euclidean, indices = nbs.kneighbors(train_X)

    # 计算每个标签列中的少数类标签
    minlabels = (np.sum(train_y == 1, axis=0) <= np.sum(train_y == 0, axis=0)).astype(int)

    # 计算邻居标签一致性 C
    C = np.array([
        [np.mean(train_y[indices[i, 1:], j] != train_y[i, j]) if train_y[i, j] == minlabels[j] else 0
         for j in range(train_y.shape[1])] for i in range(train_y.shape[0])
    ])

    # 计算样本权重 W
    W = np.sum(np.where(C < 1, C / np.sum(C[C < 1], axis=0, keepdims=True), 0), axis=1)

    # 过采样样本数
    n_sample = int(train_X.shape[0] * p)
    new_X = np.zeros((n_sample, train_X.shape[1]))
    new_y = np.zeros((n_sample, train_y.shape[1]))

    # 确定实例类型
    InstanceType = np.zeros_like(C)
    thresholds = [(0.3, LocalType.Safe), (0.7, LocalType.BoderLine), (1, LocalType.Rare)]
    
    for j in range(train_y.shape[1]):
        InstanceType[:, j] = np.select(
            [C[:, j] < t[0] for t in thresholds] + [C[:, j] >= 1],
            [t[1].value for t in thresholds] + [LocalType.Outlier.value]
        )

    seed_reference_pairs = []
    label_mismatches = []

    # 采样并生成新数据
    SumW = np.sum(W)
    for i in range(n_sample):
        random_count = np.random.random() * SumW
        seed = np.searchsorted(np.cumsum(W), random_count)

        reference = np.random.choice(indices[seed, 1:])
        ratio = np.random.random()
        seed_reference_pairs.append([seed, reference])

        # 生成新样本的特征
        new_X[i] = train_X[seed] + ratio * (train_X[reference] - train_X[seed])

        # 计算新样本的标签
        diff = []
        for j in range(train_y.shape[1]):
            npseed, npreference = train_y[seed, j], train_y[reference, j]
            dist1, dist2 = np.linalg.norm(new_X[i] - train_X[seed]), np.linalg.norm(new_X[i] - train_X[reference])
            cd = dist1 / (dist1 + dist2)

            if npseed == npreference:
                new_y[i, j] = npseed
            else:
                theta = {LocalType.Safe.value: 0.5, LocalType.BoderLine.value: 0.75, LocalType.Rare.value: 1 + 1e-5,
                         LocalType.Outlier.value: 0 - 1e-5}.get(InstanceType[seed, j], 0.5)
                if InstanceType[seed, j] == LocalType.Majority.value:
                    npseed, npreference = npreference, npseed
                    cd = 1 - cd
                new_y[i, j] = npseed if cd <= theta else npreference

                if npseed != npreference:
                    diff.append(j)
        label_mismatches.append(diff)

    combined_X = np.vstack([train_X, new_X])
    combined_y = np.vstack([train_y, new_y])

    return combined_X, combined_y, seed_reference_pairs, label_mismatches
def Random_Copy(df1, df2, p):
    SamplesToClone = int(df1.shape[0] * p)
    sub_index = list()
    ImbalanceRatioMatrix, MeanIR, countmatrix = Imbalance(np.array(df2))
    MinLabeldic, _ = Labeltype(np.array(df2))
    
    # 将 df1 和 df2 初始化为 numpy 数组
    ML_ROS_new_X = np.copy(df1)
    ML_ROS_target = np.copy(df2)
    MinLabelindex = list(MinLabeldic.keys())
    tem = ['True'] * len(MinLabelindex)
    dic = dict(zip(MinLabelindex, tem))
    minlabels = []

    # 遍历每个标签列，计算标签1和0的数量
    for i in range(df2.shape[1]):
        count0 = 0
        count1 = 0
        for j in range(df2.shape[0]):
            if df2[j, i] == 1:
                count1 += 1
            else:
                count0 += 1
        if count1 <= count0:
            minlabels.append(1)
        else:
            minlabels.append(0)

    while SamplesToClone > 0:
        for tail_label_index in MinLabelindex:
            if dic[tail_label_index] == 'False':
                continue
            
            # 确保 MinLabeldic 返回整数索引
            label_column_index = MinLabeldic[tail_label_index]

            new_IRLbl = max(countmatrix) / countmatrix[tail_label_index]
            if new_IRLbl <= MeanIR:
                dic[tail_label_index] = 'False'
                continue

            # 使用正确的列索引找到满足条件的子集
            sub_index = np.where(df2[:, tail_label_index] == 1)[0]
            randomindex = np.random.choice(sub_index)

            for index in MinLabelindex:
                if df2[randomindex, index] == minlabels[tail_label_index]:
                    countmatrix[index] += 1

            # 使用 np.vstack 来扩展数组
            ML_ROS_new_X = np.vstack([ML_ROS_new_X, df1[randomindex]])
            ML_ROS_target = np.vstack([ML_ROS_target, df2[randomindex]])
            SamplesToClone -= 1

            if SamplesToClone <= 0:
                break

        if set(dic.values()) == {'False'}:
            break

    return ML_ROS_new_X, ML_ROS_target
    
def NN(df1):
    nbs=NearestNeighbors(n_neighbors=df1.shape[0],metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean,indices= nbs.kneighbors(df1)
    return euclidean,indices

def seed_construct(df1, df2, MinLabelindex):
    S_s_all_dic = {}
    S_c_all_dic = {}
    S_s_all = []
    S_c_all = []
    S_in = []
    euclidean, indices = NN(df1)
    
    for label in range(df2.shape[1]):
        maj_index = np.where(df2[:, label] == 0)[0]  # Get indices where df2[:, label] is 0 (majority class)
        min_index = np.where(df2[:, label] == 1)[0]  # Get indices where df2[:, label] is 1 (minority class)
        
        S_s = []
        S_c = []
        
        for i in maj_index:
            top_indices = []
            for j in indices[i, 1:]:
                if df2[j, label] == 1:
                    top_indices.append(j)
                if len(top_indices) == 3:
                    break
            S_s = list(set(S_s).union(set(top_indices)))
        
        for i in S_s:
            top_indices = []
            for j in indices[i, 1:]:
                if df2[j, label] == 0:
                    top_indices.append(j)
                if len(top_indices) == 3:
                    break
            S_c = list(set(S_c).union(set(top_indices)))
        
        if label in MinLabelindex:
            S_s_all_dic[label] = S_s
            S_s_all = list(set(S_s_all).union(set(S_s)))
        
        S_c_all_dic[label] = S_c
        S_s_tmp = list(set(S_s_all).union(set(S_s)))
        S_c_all = list(set(S_c_all).union(set(S_c)))
    
    set3 = set(S_in)
    S_in = list(set3 - set(S_s_tmp + S_s_all))
    
    return S_s_all, S_c_all, S_in, S_s_all_dic, S_c_all_dic

def Get_P_s(df1, df2, MinLabelindex, S_s_all, S_s_all_dic):
    euclidean, indices = NN(df1)
    I_s = np.zeros((len(S_s_all), len(MinLabelindex)))
    
    for i in S_s_all:
        for tail_label in MinLabelindex:
            maj_index = np.where(df2[:, tail_label] == 0)[0]  # Majority class
            min_index = np.where(df2[:, tail_label] == 1)[0]  # Minority class
            
            if i in S_s_all_dic[tail_label]:  
                distances_maj = []    
                for j in indices[i, 1:]:
                    if df2[j, tail_label] == 0:
                        distances_maj.append(euclidean[i, j])
                    if len(distances_maj) == 5:
                        break
                distance_sum_maj = sum(distances_maj)
                
                distances_min = []    
                for j in indices[i, 1:]:
                    if df2[j, tail_label] == 1:
                        distances_min.append(euclidean[i, j])
                    if len(distances_min) == 5:
                        break
                distance_sum_min = sum(distances_min)
                
                I_s[S_s_all.index(i), MinLabelindex.index(tail_label)] = distance_sum_min / distance_sum_maj
    
    I_s_norm = (I_s - I_s.min(axis=0)) / (I_s.max(axis=0) - I_s.min(axis=0))
    I_s_norm_adjust = np.zeros_like(I_s_norm)
    n_k_all = []

    for label in S_s_all_dic.keys():
        min_index = len(np.where(df2[:, label] == 1)[0])  # Count of minority samples for each label
        n_k_all.append(min_index)

    for i in range(I_s_norm.shape[0]):
        for j in range(I_s_norm.shape[1]):
            n_j = len(np.where(df2[:, MinLabelindex[j]] == 1)[0])
            I_s_norm_adjust[i, j] = I_s_norm[i, j] / (1 + (min(n_k_all)) / (max(n_k_all) - min(n_k_all)))
    
    W_s = np.max(I_s_norm_adjust, axis=1)
    P_s = W_s / np.sum(W_s)
    
    return P_s

def Get_P_c(df1, df2, MinLabelindex, S_c_all, S_c_all_dic, thc):
    I_c = np.zeros((len(S_c_all), df2.shape[1]))
    euclidean, indices = NN(df1)
    
    for i in S_c_all:
        for label in range(df2.shape[1]):
            maj_index = np.where(df2[:, label] == 0)[0]  # Majority class
            min_index = np.where(df2[:, label] == 1)[0]  # Minority class
            
            if i in S_c_all_dic[label]:
                distances_maj = []    
                for j in indices[i, 1:]:
                    if df2[j, label] == 0:
                        distances_maj.append(euclidean[i, j])
                    if len(distances_maj) == 5:
                        break
                distance_sum_maj = sum(distances_maj)
                
                for j in indices[i, 1:]:
                    if df2[j, label] == 1:
                        z_indices = j
                        break
        
                distances_min = []    
                for j in indices[z_indices, 1:]:
                    if df2[j, label] == 1:
                        distances_min.append(euclidean[i, j])
                    if len(distances_min) == 5:
                        break
                distance_sum_min = sum(distances_min)  

                I_c[S_c_all.index(i), label] = distance_sum_maj / distance_sum_min
    
    I_c_min = np.min(np.ma.masked_where(I_c == 0, I_c), axis=1)
    W_c = np.where(I_c_min < thc, 0, I_c_min)
    P_c = W_c / sum(W_c)
    W_c -= 1
    gen_N_c = sum(W_c)
    
    return P_c, gen_N_c

def SynIns(S, p, euclidean, indices, genins, dfX, dfy):
    # Initialize new synthetic samples (new_X) and their corresponding target labels (target)
    new_X = np.zeros((int(genins), dfX.shape[1]))
    target = np.zeros((int(genins), dfy.shape[1]))

    for count in range(int(genins)):
        # Choose a seed sample based on probabilities
        seed_p = np.random.choice(len(p), p=p)
        seed = S[seed_p]
        
        # Find a reference sample based on the nearest neighbors
        reference = np.random.choice(indices[seed, 1:])

        # If the reference index is out of bounds, continue to the next iteration
        if reference >= dfX.shape[0]:
            continue
        
        # Generate the synthetic sample by interpolating features
        for j in range(dfX.shape[1]):
            ratio = np.random.random()
            if feature_names[j][1] == 'NUMERIC':  # Check if the feature is numeric
                new_X[count, j] = dfX[seed, j] + ratio * (dfX[reference, j] - dfX[seed, j])
            elif feature_names[j][1] in [['YES', 'NO'], ['0', '1']]:  # For categorical features
                rmd = np.random.choice([True, False])
                if rmd:
                    new_X[count, j] = dfX[seed, j]
                else:
                    new_X[count, j] = dfX[reference, j]
            else:
                new_X[count, j] = dfX[seed, j]  # Copy for other types (e.g., string)

        # Assign target labels based on the synthetic sample generation
        for j in range(dfy.shape[1]):
            target[count, j] = dfy[seed, j]

    return new_X, target

def CalcuNN1(df1, n_neighbor, df_new=None):
    nbs = NearestNeighbors(n_neighbors=n_neighbor, metric='euclidean', algorithm='kd_tree').fit(df1)
    euclidean, indices = nbs.kneighbors(df1)

    if df_new is not None:
        euclidean_new, indices_new = nbs.kneighbors(df_new)
        euclidean = np.concatenate((euclidean, euclidean_new), axis=0)
        indices = np.concatenate((indices, indices_new), axis=0)
    
    return euclidean, indices  

def Same_with_Seed(df1, df2):
    new_X = df1.copy()
    new_y = df2.copy()

    euclidean, indices = CalcuNN1(new_X, 5 + 1, df_new=None)

    MinLabeldic, MajLabeldic = Labeltype(new_y)  # Pass as NumPy arrays
    ImbalanceRatioMatrix, MeanIR, countmatrix = Imbalance(new_y)  # Pass as NumPy arrays
    MinLabelindex = list(MinLabeldic.keys())

    S_s_all, S_c_all, S_in, S_s_all_dic, S_c_all_dic = seed_construct(new_X, new_y, MinLabelindex)

    gen_N_s = 10 * len(S_s_all)
    for i in range(1):
        P_s = Get_P_s(df1, df2, MinLabelindex, S_s_all, S_s_all_dic)

        df_syn, new_target = SynIns(S_s_all, P_s, euclidean, indices, gen_N_s, new_X, new_y)
        euclidean, indices = CalcuNN1(new_X, 5 + 1, df_new=df_syn)

        new_X = np.concatenate([new_X, df_syn], axis=0)
        new_y = np.concatenate([new_y, new_target], axis=0)

        P_c, gen_N_c = Get_P_c(df1, df2, MinLabelindex, S_c_all, S_c_all_dic, 0.01)
        gen_N_c = min(int(gen_N_c), int(10 * df1.shape[0] / 1))

        if P_c.shape == (0,):
            break
        if gen_N_c != 0 and gen_N_c > 0:
            df_syn, new_target = SynIns(S_c_all, P_c, euclidean, indices, gen_N_c, new_X, new_y)
            euclidean, indices = CalcuNN1(new_X, 5 + 1, df_new=df_syn)

            new_X = np.concatenate([new_X, df_syn], axis=0)
            new_y = np.concatenate([new_y, new_target], axis=0)
        else:
            break

    n_all = [len(new_y[new_y[:, label] == 1]) for label in range(new_y.shape[1])]
    n_ave = sum(n_all) / len(n_all)
    while True:
        n_all = [len(new_y[new_y[:, label] == 1]) for label in range(new_y.shape[1])]
        gen_in = [max(n_ave - n_all[j], 0) for j in range(len(n_all))]

        if max(gen_in) == 0:
            break

        min_value = min(gen_in)
        min_label = gen_in.index(min_value)
        label_index = list(np.where(df2[:, min_label] == 1)[0])
        relevant = list(set(S_in) & set(label_index))
        if len(S_in) == 0:
            break
        P_in = np.full(len(relevant), 1 / len(relevant))
        if P_in != 0:
            df_syn, new_target = SynIns(relevant, P_in, euclidean, indices, gen_in[min_label], new_X, new_y)
            euclidean, indices = CalcuNN1(new_X, 5 + 1, df_new=df_syn)

            new_X = np.concatenate([new_X, df_syn], axis=0)
            new_y = np.concatenate([new_y, new_target], axis=0)
        else:
            break

    return new_X, new_y

In [6]:
def compute_positive_sample_ratios(y):
    """
    计算每个类的正样本比率.

    参数:
    - y: 真实标签 (形状: [n, num_labels])

    返回:
    - rc: 每个类的正样本比率 (形状: [num_labels])
    """
    
    n, q = y.shape
    
    positive_counts = np.sum(y, axis=0)  # 形状: [q]
    
    total_counts = n * np.ones(q)  # 形状: [q]
    
    # 计算正样本比率 rc
    r_c = positive_counts / total_counts
    
    return r_c

class PLM:
    def __init__(self, r_c, lambda_param):
        self.r_c = r_c
        self.lambda_param = lambda_param

    def update_ratios(self, labels, predictions):
        divergences_plus = []
        divergences_minus = []
        for c in range(labels.shape[1]):
            # Get indices of positive and negative samples
            positive_indices = np.where(labels[:, c] == 1)[0]
            negative_indices = np.where(labels[:, c] == 0)[0]    
            positive_indices_p = np.where(predictions[:, c] >= 0.5)[0]
            negative_indices_p = np.where(predictions[:, c] < 0.5)[0]
            
            # Define the sets for positive and negative samples
            S_c_plus = labels[positive_indices, c]  
            S_hat_c_plus = predictions[positive_indices_p, c]  
            S_c_minus = labels[negative_indices, c]  
            S_hat_c_minus = predictions[negative_indices_p, c] 

            # Compute probability distributions
            P_c_plus_hist, _ = np.histogram(S_c_plus, bins=5, range=(0, 1), density=True)
            P_c_minus_hist, _ = np.histogram(S_c_minus, bins=5, range=(0, 1), density=True)     
            P_hat_c_plus_hist, _ = np.histogram(S_hat_c_plus, bins=5, range=(0, 1), density=True)
            P_hat_c_minus_hist, _ = np.histogram(S_hat_c_minus, bins=5, range=(0, 1), density=True)

            # Prevent division by zero
            P_c_plus_hist += 1e-10
            P_c_minus_hist += 1e-10       
            P_hat_c_plus_hist += 1e-10
            P_hat_c_minus_hist += 1e-10

            # Compute KL divergence
            D_c_plus = entropy(P_hat_c_plus_hist, P_c_plus_hist)
            D_c_minus = entropy(P_hat_c_minus_hist, P_c_minus_hist)

            divergences_plus.append(D_c_plus)
            divergences_minus.append(D_c_minus)

        # Normalize divergences
        mu_plus = np.mean(divergences_plus)
        sigma_plus = np.std(divergences_plus)

        mu_minus = np.mean(divergences_minus)
        sigma_minus = np.std(divergences_minus)

        normalized_plus = [(d - mu_plus) / sigma_plus for d in divergences_plus]
        normalized_minus = [(d - mu_minus) / sigma_minus for d in divergences_minus]

        # Update class ratios
        for c in range(len(self.r_c)):
            D_c = normalized_plus[c] - normalized_minus[c]
            self.r_c[c] *= np.exp(self.lambda_param * D_c)

        return self.r_c
    
def update_H(H, y_pred, ids, max_history_length=5):
    y_pred_numpy = y_pred.detach().cpu().numpy()
    for i, idx in enumerate(ids):
        if idx not in H:
            H[idx] = deque(maxlen=max_history_length) 
        H[idx].append(y_pred_numpy[i])  
    return H

def update_E(H, E, ids, label_dim):
    """
    Update the E matrix based on the current predictions history in H.
    """
    for idx in ids:
        current_predictions_history = H[idx]
        
        # Check if the history is empty
        if len(current_predictions_history) == 0:
            # Skip or initialize a default value for E[idx]
            E[idx, :] = np.zeros(label_dim)  # You can also set another default value if needed
            continue
        
        last_row_index = len(current_predictions_history) - 1
        
        for j in range(label_dim):
            # Calculate E[idx, j] only if the history has valid predictions
            p = current_predictions_history[last_row_index][j]
            
            # Avoid log(0) issues by adding a small epsilon
            epsilon = 1e-10
            p = np.clip(p, epsilon, 1 - epsilon)  
            
            E[idx, j] = -1 / np.log(2) * (p * np.log(p) + (1 - p) * np.log(1 - p))
    
    return E

def update_dataset_label(E, dataset, ids, label_dim,threshold):
    # 获取 dataset 中的标签 (y) 并将其转换为 numpy 数组
    y = dataset.tensors[1].cpu().numpy()
    # 遍历样本，从 start 索引开始
    for i in ids:
        for j in range(label_dim):  # 遍历每个标签
            if E[i, j] >= threshold:  # 判断 E 矩阵的值是否小于等于阈值
                # 如果当前标签为 0，则将其更新为 1，反之为 0
                y[i, j] = 1 if y[i, j] == 0 else 0

    # 将更新后的 y 重新转换为 tensor，并封装回 TensorDataset 中
    updated_y = torch.tensor(y, dtype=torch.float, device=dataset.tensors[1].device)
    
    # 保留原始的输入特征，更新后的标签重新封装为 TensorDataset
    updated_dataset = TensorDataset(dataset.tensors[0], updated_y)

    return updated_dataset
        
class CustomDataLoader:
    def __init__(self, dataset, net, configs, H, E, warm_epoch, origin_flags):
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.original_indices = list(range(len(self.dataset)))  # 保持原始索引
        self.dataset_indices = self.original_indices.copy()  # 用于迭代的数据索引
        self.origin_flags = origin_flags  # 标记每条数据是否来自原始数据集
        self.warm_epoch = warm_epoch
        self.H = H
        self.E = E
        self.current_epoch = 0  # 初始化当前 epoch
        self.shuffle = True  # 是否打乱数据

    def __iter__(self):
        """迭代返回数据的批次"""
        # 如果需要打乱数据，先打乱 dataset_indices
        if self.shuffle:
            random.shuffle(self.dataset_indices)

        # 按批次索引切片遍历数据
        for start_idx in range(0, len(self.dataset_indices), self.batch_size):
            end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
            batch_indices = self.dataset_indices[start_idx:end_idx]

            # 返回原始索引, 数据 (x, y) 和标志位
            yield self.get_data_from_indices(batch_indices)

    def get_data_from_indices(self, indices):
        """根据索引返回对应的数据"""
        # 使用原始索引获取数据
        x, y = zip(*[self.dataset[self.original_indices[i]] for i in indices])
        return indices, torch.stack(x), torch.stack(y)

    def set_epoch(self, epoch):
        """设置当前的 epoch"""
        self.current_epoch = epoch
        # 在每个 epoch 开始时打乱数据
        if self.shuffle:
            random.shuffle(self.dataset_indices)

In [7]:
def training(configs, warm_epoch, r_c, lambda_param, threshold, origin_flags):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = DELA(configs).to(device)
    num_epochs = configs['epoch']
    batch_size = configs['batch_size']
    
    # 优化器和超参数设置
    lr = 1e-4
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
    writer = SummaryWriter(comment=f'{dataname}')

    # 初始化数据集的H和E
    ins_dim = len(origin_flags)
    true_count = origin_flags.count(True)
    generate_idx = [i for i in range(ins_dim) if true_count <= i <= ins_dim]
    
    label_dim = configs['num_classes']
    sample_indices = list(range(ins_dim))
    H = {idx: deque(maxlen=5) for idx in sample_indices}
    E = np.zeros((ins_dim, label_dim))
    
    
    
    # 自定义数据加载器
    custom_dataloader = CustomDataLoader(train_dataset, net=net, configs=configs, H=H, E=E, warm_epoch=warm_epoch, origin_flags=origin_flags)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # 初始化训练状态
    best_auc = 0
    best_model_state = None
    global_step = 0
    warmup_steps = warm_epoch * int(ins_dim / batch_size)
    plm = PLM(r_c=r_c, lambda_param=lambda_param)
    
    # 学习率更新函数
    def update_learning_rate(optimizer, global_step, warmup_steps, base_lr):
        lr = base_lr * (global_step / warmup_steps) if global_step < warmup_steps else base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    for epoch in range(num_epochs + 1):
        net.train()
        all_y_pred = []
        all_y_true = []
        custom_dataloader.set_epoch(epoch)
        
        # 批次训练
        for idx, x, y in custom_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = net(x)
            
            # 损失计算与反向传播
            loss_dict = net.loss_function_train(outputs, y, r_c,epoch)
            loss = loss_dict['Loss']
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step, warmup_steps, lr)
            
            if idx in generate_idx:
                custom_dataloader.H = update_H(custom_dataloader.H, outputs[configs['out_index']], idx)
            
            # 二值化预测结果
#             binary_outputs = torch.where(outputs[configs['out_index']] >= 0.5, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))
            all_y_pred.append(outputs[configs['out_index']].detach())
            all_y_true.append(y.detach())

        # 合并所有批次的数据
        all_y_pred = torch.cat(all_y_pred, dim=0)
        all_y_true = torch.cat(all_y_true, dim=0)
        
        # 更新 r_c
        r_c = plm.update_ratios(all_y_true.cpu().numpy(), all_y_pred.cpu().numpy())
        
        
        # 更新H、E以及数据集标签       
        custom_dataloader.E = update_E(custom_dataloader.H, custom_dataloader.E, generate_idx, label_dim)
        custom_dataloader.dataset = update_dataset_label(custom_dataloader.E, custom_dataloader.dataset, generate_idx, label_dim,threshold)
        
        writer.add_scalar('train/loss', loss.item(), epoch)
        
        # 评估模型
        auc, best_model_state = evaluate_model(net, validation_dataloader, best_auc, best_model_state, epoch)
    
    # 使用最优模型状态进行测试
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['batch_size'], device)
    
    return mets
def evaluate_model(net, dataloader, best_auc, best_model_state, epoch):
    net.eval()
    y_true_list = []
    y_scores_list = []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            outputs = net(x)
            y_true_list.append(y.cpu().numpy())
            y_scores_list.append(outputs[configs['out_index']].cpu().numpy())
    
    y_true = np.vstack(y_true_list)
    y_scores = np.vstack(y_scores_list)
    
    auc = macro_averaging_auc(y_true, y_scores, y_scores)
    
    if auc > best_auc:
        best_auc = auc
        best_model_state = net.state_dict().copy()
    
    return auc, best_model_state

In [7]:
path_to_arff_files = ["scene","yeast", 'enron','cal500']
label_counts = [6,14,53,174]
select_feature=[1,1,1,1]
# sp=[0.1,0.2,0.3,0.5,0.7]
sp=[0]
path_to_arff_files = ["scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1",'cal500']
label_counts = [ 6,14,374,101,101,101,28,25,174]
select_feature=[1,1,1,0.02,0.02,0.02,0.05,0.05,1]
for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    # Load data and select features/labels
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[idx],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
    )
    
    X, feature_names = FeatureSelect(X, select_feature[idx])  
    y, label_names = LabelSelect(y)
    print(f"Processing dataset: {dataname}")
    # Scale the features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Set parameters for training
    warm_epoch = 5

    # K-Fold cross validation setup
    k_fold = IterativeStratification(n_splits=5, order=1, random_state=42)
    dicts = []
    for i in sp:
        p =i  # oversampling ratio
        print(f"samplingrate: {p}")
        # Iterate through k-fold splits
        for fold_idx, (train_idx, test_idx) in enumerate(k_fold.split(X, y)):
            # Compute positive sample ratios for the training set
            # Perform sampling on the training set
            train_X, train_y = X[train_idx], y[train_idx]
#             train_X, train_y, seed_reference_pairs, label_mismatches = train_set_weight_and_oversampling(train_X, train_y, p)
#             train_X, train_y = train_set_weight_and_oversampling(train_X, train_y, p)
            train_X, train_y = Same_with_Seed(train_X, train_y)
            r_c = compute_positive_sample_ratios(train_y)
            # Create the TensorDataset for train and test sets
            train_dataset = TensorDataset(
                torch.tensor(train_X, device=device, dtype=torch.float),
                torch.tensor(train_y, device=device, dtype=torch.float)
            )

            test_dataset = TensorDataset(
                torch.tensor(X[test_idx], device=device, dtype=torch.float),
                torch.tensor(y[test_idx], device=device, dtype=torch.float)
            )

            # Create flags for original data size and oversampled data
            original_data_size = len(train_idx)
            new_data_size = int(original_data_size * p)
            origin_flags = [True] * original_data_size + [False] * new_data_size

            # Generate configurations for training
            configs = CFG('DELA', X[train_idx], y[train_idx]).getconfig()

            # Perform training
            result_dict = training(configs, warm_epoch, r_c, 0.01, 0.3, origin_flags)
            dicts.append(result_dict)

        # Calculate averages and standard deviations across folds
        averages_and_stds = {}
        for key in dicts[0].keys():
            values = [d[key] for d in dicts]
            averages_and_stds[key] = {
                'average': round(np.mean(values), 4),
                'std': round(np.std(values), 4)
            }

        # Print the results for the current dataset
        print(f"Results for {dataname}:")
        print(averages_and_stds)

Processing dataset: scene
samplingrate: 0


NameError: name 'training' is not defined

In [7]:
def training(configs, warm_epoch, r_c, lambda_param, threshold, origin_flags):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = PACA(configs).to(device)
    num_epochs = configs['epoch']
    batch_size = configs['batch_size']
    
    # 优化器和超参数设置
    lr = 1e-4
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
    writer = SummaryWriter(comment=f'{dataname}')

    # 初始化数据集的H和E
    ins_dim = len(origin_flags)
    true_count = origin_flags.count(True)
    generate_idx = [i for i in range(ins_dim) if true_count <= i <= ins_dim]
    
    label_dim = configs['num_classes']
    sample_indices = list(range(ins_dim))
    H = {idx: deque(maxlen=5) for idx in sample_indices}
    E = np.zeros((ins_dim, label_dim))
    
    
    
    # 自定义数据加载器
    custom_dataloader = CustomDataLoader(train_dataset, net=net, configs=configs, H=H, E=E, warm_epoch=warm_epoch, origin_flags=origin_flags)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # 初始化训练状态
    best_auc = 0
    best_model_state = None
    global_step = 0
    warmup_steps = warm_epoch * int(ins_dim / batch_size)
    plm = PLM(r_c=r_c, lambda_param=lambda_param)
    
    # 学习率更新函数
    def update_learning_rate(optimizer, global_step, warmup_steps, base_lr):
        lr = base_lr * (global_step / warmup_steps) if global_step < warmup_steps else base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    for epoch in range(num_epochs + 1):
        net.train()
        all_y_pred = []
        all_y_true = []
        custom_dataloader.set_epoch(epoch)
        
        # 批次训练
        for idx, x, y in custom_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = net(x,y)
            
            # 损失计算与反向传播
            loss_dict = net.loss_function_train(outputs, y, r_c,epoch)
            loss = loss_dict['Loss']
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step, warmup_steps, lr)
            
            if idx in generate_idx:
                custom_dataloader.H = update_H(custom_dataloader.H, outputs[configs['out_index']], idx)
            
            # 二值化预测结果
#             binary_outputs = torch.where(outputs[configs['out_index']] >= 0.5, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))
            all_y_pred.append(outputs[configs['out_index']].detach())
            all_y_true.append(y.detach())

        # 合并所有批次的数据
        all_y_pred = torch.cat(all_y_pred, dim=0)
        all_y_true = torch.cat(all_y_true, dim=0)
        
        # 更新 r_c
        r_c = plm.update_ratios(all_y_true.cpu().numpy(), all_y_pred.cpu().numpy())
        
        
        # 更新H、E以及数据集标签       
        custom_dataloader.E = update_E(custom_dataloader.H, custom_dataloader.E, generate_idx, label_dim)
        custom_dataloader.dataset = update_dataset_label(custom_dataloader.E, custom_dataloader.dataset, generate_idx, label_dim,threshold)
        
        writer.add_scalar('train/loss', loss.item(), epoch)
        
        # 评估模型
        auc, best_model_state = evaluate_model(net, validation_dataloader, best_auc, best_model_state, epoch)
    
    # 使用最优模型状态进行测试
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['batch_size'], device)
    
    return mets
def evaluate_model(net, dataloader, best_auc, best_model_state, epoch):
    net.eval()
    y_true_list = []
    y_scores_list = []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            outputs = net(x,y)
            y_true_list.append(y.cpu().numpy())
            y_scores_list.append(outputs[configs['out_index']].cpu().numpy())
    
    y_true = np.vstack(y_true_list)
    y_scores = np.vstack(y_scores_list)
    
    auc = macro_averaging_auc(y_true, y_scores, y_scores)
    
    if auc > best_auc:
        best_auc = auc
        best_model_state = net.state_dict().copy()
    
    return auc, best_model_state

In [ ]:
path_to_arff_files = ["scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1",'cal500','Corel16k','tmc2007-500','Eurlex-sm']
label_counts = [ 6,14,374,101,101,101,28,25,174,153,22,201]
select_feature=[1,1,1,0.02,0.02,0.02,0.05,0.05,1,1,1,0.1]
sp=[0.1,0.2,0.3,0.5,0.7]

path_to_arff_files = ["rcv1subset3","yahoo-Business1","yahoo-Arts1",'cal500','Corel16k','tmc2007-500','Eurlex-sm']
label_counts = [101,28,25,174,153,22,201]
select_feature=[0.02,0.05,0.05,1,1,1,0.1]
sp=[0.1,0.2,0.3,0.5,0.7]
for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    # Load data and select features/labels
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[idx],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
    )
    
    X, feature_names = FeatureSelect(X, select_feature[idx])  
    y, label_names = LabelSelect(y)
    print(f"Processing dataset: {dataname}")
    # Scale the features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Set parameters for training
    warm_epoch = 5

    # K-Fold cross validation setup
    k_fold = IterativeStratification(n_splits=5, order=1, random_state=42)
    dicts = []
    for i in sp:
        p =i  # oversampling ratio
        print(f"samplingrate: {p}")
        # Iterate through k-fold splits
        for fold_idx, (train_idx, test_idx) in enumerate(k_fold.split(X, y)):
            # Compute positive sample ratios for the training set
            # Perform sampling on the training set
            train_X, train_y = X[train_idx], y[train_idx]
#             train_X, train_y, seed_reference_pairs, label_mismatches = train_set_weight_and_oversampling(train_X, train_y, p)
            train_X, train_y = Random_Copy(train_X, train_y, p)
#             train_X, train_y = Same_with_Seed(train_X, train_y)
            r_c = compute_positive_sample_ratios(train_y)
            # Create the TensorDataset for train and test sets
            train_dataset = TensorDataset(
                torch.tensor(train_X, device=device, dtype=torch.float),
                torch.tensor(train_y, device=device, dtype=torch.float)
            )

            test_dataset = TensorDataset(
                torch.tensor(X[test_idx], device=device, dtype=torch.float),
                torch.tensor(y[test_idx], device=device, dtype=torch.float)
            )

            # Create flags for original data size and oversampled data
            original_data_size = len(train_idx)
            new_data_size = int(original_data_size * p)
            origin_flags = [True] * original_data_size + [False] * new_data_size

            # Generate configurations for training
            configs = CFG('PACA', X[train_idx], y[train_idx]).getconfig()

            # Perform training
            result_dict = training(configs, warm_epoch, r_c, 0.01, 0.3, origin_flags)
            dicts.append(result_dict)

        # Calculate averages and standard deviations across folds
        averages_and_stds = {}
        for key in dicts[0].keys():
            values = [d[key] for d in dicts]
            averages_and_stds[key] = {
                'average': round(np.mean(values), 4),
                'std': round(np.std(values), 4)
            }

        # Print the results for the current dataset
        print(f"Results for {dataname}:")
        print(averages_and_stds)

Processing dataset: rcv1subset3
samplingrate: 0.1
